# Example DeepResearchAgent

In [1]:
import os
import time
from os.path import join, exists
from os import listdir, makedirs
from datetime import datetime
from google import genai
from google.genai import types
from openai import OpenAI
import requests
import json
from pydantic import BaseModel, Field
from crawl4ai import *
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel
from dataclasses import dataclass

import asyncio
import nest_asyncio 
# Add this line to allow nested event loops
nest_asyncio.apply()

from agent_tools import *
from agent_utils import *
from rich import print as rprint
from rich.console import Console
from rich.markdown import Markdown
config = Config()

### Crawl Webpages

In [ ]:
query_folder = "test-time-compute"
# Create folder if not exists
if not exists(query_folder):
    makedirs(query_folder)

document_data = {}

for result in json_response['organic']:
    title = result['title']
    markdown = crawl_website(result['link'])
    filename = result['title'] + ".md"

    document_data[title] = {
        'topic': topic,
        'link': result['link'],
        'snippet': result['snippet'],
        'date': result['date'],
        'position': result['position'],
        'markdown': markdown,
        'filename': filename
    }

    with open(join(query_folder, filename), "w") as f:
        f.write(markdown)

### Document Quality Assessment

In [28]:
class DocumentQuality(BaseModel):
    filename: str = Field(description="The name of the file")   
    document_length: int = Field(description="The length of the document (0 to 10). Is the document short or long.", ge=0, le=10)
    relevance: int = Field(description="How relevant is the document with the main topic (0 to 10).", ge=0, le=10)
    document_quality: int = Field(description="Guess the quality of the document (0 to 10).", ge=0, le=10)
    document_age: int = Field(description="The age of the document relative to the current data (0 to 10).", ge=0, le=10)
    additional_observations: str = Field(description="If you noticed something strange about the document. Write it in form of an instruction for another LLM agents.")

def rate_document(document):

    system_instruction = f"""
    You are an professional scientific journalist. 

    You will receive research related documents (markdown format). 
    Your goal is to estimate the relevance and quality of this document (based on a given topic).
    The document will later be used for writing a reasearch report/document.
    If the quality of the text isn't good, this will lead to an overall bad outcome of the report. 

    Topic: {document['topic']}    
    Current Date: {datetime.now().date()}
    Document Date: {document['date']}
    Link: {document['link']}
    """

    markdown_content = markdown

    response = client.models.generate_content(
        model=model_name,
        contents=markdown_content,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=DocumentQuality,
            system_instruction=system_instruction,
            temperature=0.3,
        ),
    )
    document['response'] = response
    return document

### Process Files (Quality Assessment)

In [ ]:
# Create folder if not exists
if exists(query_folder):
    # Get all files in folder
    files = listdir(query_folder)

for title, document in document_data.items():
    print(f"File: {document.filename}")
    document = rate_document(document)
    print(f"Reponse Text: {response.text}")
    document_data[title] = document
    time.sleep(5) # sleep for 5 seconds (rate limit is 10 RPM)

In [ ]:
system_instruction = """You are an AI assistant designed to produce output that is visually appealing and easily readable in a terminal. When formatting your responses, utilize the syntax of the Python `rich` library. This involves using square brackets to enclose formatting tags.
        Here are some examples of how to apply formatting:

        * **Emphasis:** Instead of "This is important", output "[bold]This is important[/]".
        * **Headers/Titles:** Instead of "Section Title:", output "[bold blue]Section Title:[/]".
        * **Warnings:** Instead of "Warning!", output "[bold red]Warning![/]".
        * **Success Messages:** Instead of "Operation successful.", output "[green]Operation successful.[/]".
        * **Lists:** You can use colors for list items like "[cyan]*[/] Item 1".

        Always use the `rich` library's syntax for formatting terminal output to enhance readability."""

response = client.models.generate_content(
    model=flash_thinking_model,
    contents="Show me the proof for the euler identity?",
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.3,
    ),
)
print(response.text)

### File Upload

In [ ]:
file = client.files.upload(path="a11.text")
response = client.models.generate_content(
    model=model_name, contents=["Summarize this file", file]
)
print(response.text)

In [26]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

/home/dplaia/Projekte/DeepResearchAgent/.venv/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


The weather in Boston is sunny.



In [ ]:
from WebSearchAgent import *
response = await google_scholar_search("RAG Benchmark", num_pages=3)
paper = response[0]
title = paper['title']
link = paper['link']
pdfUrl = paper['pdfUrl']
print(f"{title}")
print(f"    ({link.replace("https://","")}) ({pdfUrl.replace("https://","")})")

## Paper With Code

In [ ]:
pwc_title = await papers_with_code_search(title)['results'][0]['paper']['title']
pwc_pdf = papers_with_code_search(title)['results'][0]['paper']['url_pdf']
print(f"{pwc_title}  ({pwc_pdf})")

## Generic Google Search

In [ ]:
query = "OpenRouter Deepseek R1 for free"
response = await google_general_search(query)
rprint(response)
#console.print(Markdown(response))